In [2]:
import psycopg2
from psycopg2 import sql
from gsHandler import retrieve_data, create_new_sheet, create_new_page, append_row, update_row

Successfully authenticated.


## local serever connection

In [38]:
# Database configuration
DB_HOST = "localhost"
DB_NAME = "postgres"
DB_USER = "sanjananambiar"
DB_PASS = "sanjana"

# Connect to the database
def get_db_connection():
    try:
        conn = psycopg2.connect(
            host=DB_HOST,
            database=DB_NAME,
            user=DB_USER,
            password=DB_PASS
        )
        return conn
    except psycopg2.Error as e:
        print(f"Error connecting to database: {e}")
        return None

In [34]:
def create_tables():
    conn = get_db_connection()
    with conn:
        with conn.cursor() as cur:
            cur.execute("""
                CREATE TABLE IF NOT EXISTS traits_table (
                    trait_id SERIAL PRIMARY KEY,
                    trait_name VARCHAR(100) NOT NULL
                );
            """)
            
            cur.execute("""
                CREATE TABLE IF NOT EXISTS cxo_roles (
                    cxo_id SERIAL PRIMARY KEY,
                    cxo_name VARCHAR(100) NOT NULL
                );
            """)

            cur.execute("""
                CREATE TABLE IF NOT EXISTS assessment_data_model (
                    question_number INTEGER,
                    decision_statement_value VARCHAR(255),
                    trait_number INTEGER,
                    trait_name VARCHAR(100),
                    decision_statement VARCHAR(255),
                    scenario VARCHAR(255)
                );
            """)

            cur.execute("""
                CREATE TABLE IF NOT EXISTS insight_table (
                    trait1 VARCHAR(100),
                    trait1_score VARCHAR(100),
                    trait2 VARCHAR(100),
                    trait2_score VARCHAR(100),
                    insight TEXT
                );
            """)

            cur.execute("""
                CREATE TABLE IF NOT EXISTS consequence_group (
                    consequence TEXT,
                    consequence_group TEXT
                );
            """)
            
            cur.execute("""
                CREATE TABLE IF NOT EXISTS consequence_statement (
                    consequence TEXT,
                    scenario VARCHAR(255),
                    decision_statement TEXT
                );
            """)

            cur.execute("""
                CREATE TABLE IF NOT EXISTS assessment_entries (
                    assessment_id SERIAL PRIMARY KEY,
                    user_id UUID,
                    customer_centric FLOAT,
                    collaborative FLOAT,
                    agile FLOAT,
                    innovative FLOAT,
                    risk_taking FLOAT,
                    visionary FLOAT,
                    hustler FLOAT,
                    passionate FLOAT,
                    resilient FLOAT,
                    educational FLOAT,
                    analytical FLOAT,
                    frugal FLOAT,
                    legacy FLOAT,
                    digital FLOAT,
                    problem_solver FLOAT,
                    question1 VARCHAR(255),
                    question2 VARCHAR(255),
                    question3 VARCHAR(255),
                    question4 VARCHAR(255),
                    question5 VARCHAR(255),
                    question6 VARCHAR(255),
                    question7 VARCHAR(255),
                    question8 VARCHAR(255),
                    question9 VARCHAR(255),
                    question10 VARCHAR(255),
                    question11 VARCHAR(255),
                    question12 VARCHAR(255),
                    question13 VARCHAR(255),
                    question14 VARCHAR(255),
                    question15 VARCHAR(255),
                    question16 VARCHAR(255),
                    question17 VARCHAR(255),
                    question18 VARCHAR(255),
                    question19 VARCHAR(255),
                    question20 VARCHAR(255),
                    question21 VARCHAR(255),
                    question22 VARCHAR(255),
                    question23 VARCHAR(255),
                    question24 VARCHAR(255),
                    question25 VARCHAR(255),
                    question26 VARCHAR(255),
                    question27 VARCHAR(255),
                    question28 VARCHAR(255),
                    question29 VARCHAR(255),
                    question30 VARCHAR(255),
                    question31 VARCHAR(255),
                    question32 VARCHAR(255),
                    question33 VARCHAR(255),
                    question34 VARCHAR(255),
                    question35 VARCHAR(255),
                    question36 VARCHAR(255),
                    question37 VARCHAR(255),
                    question38 VARCHAR(255),
                    question39 VARCHAR(255),
                    question40 VARCHAR(255),
                    question41 VARCHAR(255),
                    question42 VARCHAR(255),
                    question43 VARCHAR(255),
                    question44 VARCHAR(255),
                    question45 VARCHAR(255),
                    submission_ip VARCHAR(255),
                    last_update_date DATE
                );
            """)

            cur.execute("""
                CREATE TABLE IF NOT EXISTS user_info (
                    user_id UUID PRIMARY KEY,
                    name VARCHAR(255),
                    email VARCHAR(255),
                    age INT,
                    linkedin_url VARCHAR(255),
                    education_level VARCHAR(255),
                    employment_status VARCHAR(255),
                    entrepreneurial_experience INT,
                    current_startup_stage VARCHAR(255),
                    number_of_startups INT,
                    role_in_entrepreneurship VARCHAR(255),
                    industry_experience VARCHAR(255),
                    number_of_previous_startups INT,
                    location VARCHAR(255),
                    gender VARCHAR(50),
                    startup_name VARCHAR(255)
                );
            """)

            cur.execute("""
                CREATE TABLE IF NOT EXISTS person_role_table (
                    person_role_id SERIAL PRIMARY KEY,
                    user_id VARCHAR(255),
                    role_id INT,
                    started_at DATE
                );
            """)

            cur.execute("""
                CREATE TABLE IF NOT EXISTS roles_table (
                    role_id SERIAL PRIMARY KEY,
                    role_name VARCHAR(255),
                    created_at DATE
                );
            """)

            cur.execute("""
                CREATE TABLE IF NOT EXISTS consequence_results (
                user_id UUID,
                name VARCHAR(255),
                delayed_product_market_fit FLOAT,
                lack_of_product_market_fit FLOAT,
                unable_to_complete_fundraise FLOAT,
                lack_of_growth FLOAT,
                lack_of_revenue FLOAT,
                high_turnover_of_talent FLOAT,
                inefficient_processes FLOAT,
                time_consuming_client_acquisition FLOAT,
                low_customer_conversion FLOAT,
                low_customer_satisfaction FLOAT,
                low_customer_retention FLOAT,
                high_cash_burnrate FLOAT,
                high_team_conflict FLOAT,
                high_key_man_risk FLOAT,
                lack_of_partnerships_collaborations FLOAT,
                lack_of_scalability FLOAT,
                lack_of_data_integrity FLOAT,
                lack_of_data_security FLOAT,
                lack_of_marketing FLOAT,
                lack_of_motivation FLOAT,
                lack_of_leadership FLOAT,
                lack_of_innovation FLOAT,
                lack_of_clarity FLOAT,
                too_much_dependency_on_external_factors FLOAT,
                lack_of_technological_advancements FLOAT,
                lack_of_unique_value_proposition FLOAT,
                lack_of_customer_variety FLOAT,
                lack_of_intellectual_property FLOAT,
                lacking_solution_quality FLOAT,
                lack_of_supporters FLOAT,
                missed_opportunities FLOAT,
                delayed_revenue FLOAT,
            );
            """)

In [35]:
create_tables()

In [30]:
# Insert initial data
def insert_initial_data():
    conn = get_db_connection()
    if conn is None:
        return
    try:
        with conn:
            with conn.cursor() as cur:
                # Insert data into traits_table
                traits = [('Customer-Centric',), ('Collaborative',), ('Agile',), ('Innovative',), ('Risk-Taking',),
                          ('Visionary',), ('Hustler',), ('Passionate',), ('Resilient',), ('Educational',), ('Analytical',),
                          ('Frugal',), ('Legacy',), ('Digital',), ('Problem Solver',)]
                cur.executemany("INSERT INTO traits_table (trait_name) VALUES (%s)", traits)

                # Insert data into cxo_roles
                cxo_roles = [('CEO',), ('CSO',), ('CPO',), ('CMO',), ('CFO',), ('CTO',), ('COO',), ('Advisor',)]
                cur.executemany("INSERT INTO cxo_roles (cxo_name) VALUES (%s)", cxo_roles)

                # Insert data into roles_table
                roles = [('Buiness Partner',), ('Mentor/Advisor',)]
                cur.executemany("INSERT INTO roles_table (role_name) VALUES (%s)", roles)
    except psycopg2.Error as e:
        print(f"Error inserting initial data: {e}")

insert_initial_data()

In [31]:
# Function to insert data from Google Sheets to PostgreSQL
def insert_data_from_google_sheets(sheet_id, sheet_names):
    conn = get_db_connection()
    if conn is None:
        return
    try:
        with conn:
            with conn.cursor() as cur:
                # Loop through each sheet and insert data
                for sheet_name in sheet_names:
                    data = retrieve_data(sheet_id, sheet_name)
                    if sheet_name == 'Assessment Form':
                        for row in data:
                            cur.execute("""
                                INSERT INTO assessment_data_model (question_number, decision_statement_value, trait_number, trait_name, decision_statement, scenario)
                                VALUES (%s, %s, %s, %s, %s, %s)
                            """, (row['Question number'], row['Decision Statement Value'], row['Trait Number'], row['Trait Name'], row['Decision Statement'], row['Scenario']))
                    elif sheet_name == 'Consequence':
                        for row in data:
                            cur.execute("""
                                INSERT INTO consequence_group (consequence, consequence_group)
                                VALUES (%s, %s)
                            """, (row['Consequence'], row['Consequence Group']))
                    elif sheet_name == 'Consequence-Statement':
                        for row in data:
                            cur.execute("""
                                INSERT INTO consequence_statement (consequence, scenario, decision_statement)
                                VALUES (%s, %s, %s)
                            """, (row['Consequence'], row['Scenario'], row['Decision Statement']))
                    elif sheet_name == 'Insight Database':
                        for row in data:
                            cur.execute("""
                                INSERT INTO insight_table (trait1, trait1_score, trait2, trait2_score, insight)
                                VALUES (%s, %s, %s, %s, %s)
                            """, (row['Trait 1'], row['Trait 1 Score'], row['Trait 2'], row['Trait 2 Score'], row['Insight']))
    except psycopg2.Error as e:
        print(f"Error inserting data from Google Sheets: {e}")

sheet_id = '1rUDTXpzyyLuHXc_Tq8lANaqxbyaOR6DOLAP4Nq7X9Q0'  
sheet_names = ['Assessment Form', 'Consequence', 'Consequence-Statement', 'Insight Database']

insert_data_from_google_sheets(sheet_id, sheet_names)

Successfully retrieved data from the sheet. Number of records: 225
Successfully retrieved data from the sheet. Number of records: 32
Successfully retrieved data from the sheet. Number of records: 327
Successfully retrieved data from the sheet. Number of records: 945


In [28]:
import pandas as pd
from datetime import datetime

#read a csv file to a dataframe 
df = pd.read_csv('raw_data_edited.csv')
df.columns

Index(['User_id', 'Name', 'Email', 'Age', 'LinkedIn URL', 'Education Level',
       'Current Employment Status',
       'Entrepreneurial Experience (Number of Years)', 'Current Startup Stage',
       'Number of Current Startups/Projects', 'Your Role in Entrepreneurship',
       'Type of Industry Experience', 'Number of Previous Startups',
       'Where are you based?', 'Gender', 'Startup Name', '1. Customer-Centric',
       '2. Collaborative', '3. Agile', '4. Innovative', '5. Risk-Taking',
       '6. Visionary', '7. Hustler', '8. Passionate', '9. Resilient',
       '10. Educational', '11. Analytical', '12. Frugal', '13. Legacy',
       '14. Digital', '15. Problem Solver', 'Question 1', 'Question 2',
       'Question 3', 'Question 4', 'Question 5', 'Question 6', 'Question 7',
       'Question 8', 'Question 9', 'Question 10', 'Question 11', 'Question 12',
       'Question 13', 'Question 14', 'Question 15', 'Question 16',
       'Question 17', 'Question 18', 'Question 19', 'Question 20',


In [31]:
# Function to insert data into user_info table
def insert_user_info(row):
    query = sql.SQL("INSERT INTO user_info (user_id, name, email, age, linkedin_url, education_level, employment_status, entrepreneurial_experience, current_startup_stage, number_of_startups, role_in_entrepreneurship, industry_experience, number_of_previous_startups, location, gender, startup_name) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s) ON CONFLICT (user_id) DO NOTHING")
    data = (
        row['User_id'],
        row['Name'] if pd.notnull(row['Name']) else None,
        row['Email'] if pd.notnull(row['Email']) else None,
        int(row['Age']) if pd.notnull(row['Age']) else None,
        row['LinkedIn URL'] if pd.notnull(row['LinkedIn URL']) else None,
        row['Education Level'] if pd.notnull(row['Education Level']) else None,
        row['Current Employment Status'] if pd.notnull(row['Current Employment Status']) else None,
        int(row['Entrepreneurial Experience (Number of Years)']) if pd.notnull(row['Entrepreneurial Experience (Number of Years)']) else None,
        row['Current Startup Stage'] if pd.notnull(row['Current Startup Stage']) else None,
        int(row['Number of Current Startups/Projects']) if pd.notnull(row['Number of Current Startups/Projects']) else None,
        row['Your Role in Entrepreneurship'] if pd.notnull(row['Your Role in Entrepreneurship']) else None,
        row['Type of Industry Experience'] if pd.notnull(row['Type of Industry Experience']) else None,
        int(row['Number of Previous Startups']) if pd.notnull(row['Number of Previous Startups']) else None,
        row['Where are you based?'] if pd.notnull(row['Where are you based?']) else None,
        row['Gender'] if pd.notnull(row['Gender']) else None,
        row['Startup Name'] if pd.notnull(row['Startup Name']) else None
    )
    conn = get_db_connection()
    if conn is None:
        return
    try:
        with conn:
            with conn.cursor() as cur:
                cur.execute(query, data)
    except Exception as e:
        print(f"Error inserting data: {e}")

# Function to insert data into assessment_entries table
def insert_assessment_entries(row):
    query = sql.SQL("""
    INSERT INTO assessment_entries (
        user_id, customer_centric, collaborative, agile, innovative, risk_taking, visionary, hustler, passionate, resilient, educational, analytical, frugal, legacy, digital, problem_solver, question1, question2, question3, question4, question5, question6, question7, question8, question9, question10, question11, question12, question13, question14, question15, question16, question17, question18, question19, question20, question21, question22, question23, question24, question25, question26, question27, question28, question29, question30, question31, question32, question33, question34, question35, question36, question37, question38, question39, question40, question41, question42, question43, question44, question45, submission_ip, last_update_date
    ) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s) 
    """)
    data = (
        row['User_id'],
        float(row['1. Customer-Centric']) if pd.notnull(row['1. Customer-Centric']) else None,
        float(row['2. Collaborative']) if pd.notnull(row['2. Collaborative']) else None,
        float(row['3. Agile']) if pd.notnull(row['3. Agile']) else None,
        float(row['4. Innovative']) if pd.notnull(row['4. Innovative']) else None,
        float(row['5. Risk-Taking']) if pd.notnull(row['5. Risk-Taking']) else None,
        float(row['6. Visionary']) if pd.notnull(row['6. Visionary']) else None,
        float(row['7. Hustler']) if pd.notnull(row['7. Hustler']) else None,
        float(row['8. Passionate']) if pd.notnull(row['8. Passionate']) else None,
        float(row['9. Resilient']) if pd.notnull(row['9. Resilient']) else None,
        float(row['10. Educational']) if pd.notnull(row['10. Educational']) else None,
        float(row['11. Analytical']) if pd.notnull(row['11. Analytical']) else None,
        float(row['12. Frugal']) if pd.notnull(row['12. Frugal']) else None,
        float(row['13. Legacy']) if pd.notnull(row['13. Legacy']) else None,
        float(row['14. Digital']) if pd.notnull(row['14. Digital']) else None,
        float(row['15. Problem Solver']) if pd.notnull(row['15. Problem Solver']) else None,
        row['Question 1'] if pd.notnull(row['Question 1']) else None,
        row['Question 2'] if pd.notnull(row['Question 2']) else None,
        row['Question 3'] if pd.notnull(row['Question 3']) else None,
        row['Question 4'] if pd.notnull(row['Question 4']) else None,
        row['Question 5'] if pd.notnull(row['Question 5']) else None,
        row['Question 6'] if pd.notnull(row['Question 6']) else None,
        row['Question 7'] if pd.notnull(row['Question 7']) else None,
        row['Question 8'] if pd.notnull(row['Question 8']) else None,
        row['Question 9'] if pd.notnull(row['Question 9']) else None,
        row['Question 10'] if pd.notnull(row['Question 10']) else None,
        row['Question 11'] if pd.notnull(row['Question 11']) else None,
        row['Question 12'] if pd.notnull(row['Question 12']) else None,
        row['Question 13'] if pd.notnull(row['Question 13']) else None,
        row['Question 14'] if pd.notnull(row['Question 14']) else None,
        row['Question 15'] if pd.notnull(row['Question 15']) else None,
        row['Question 16'] if pd.notnull(row['Question 16']) else None,
        row['Question 17'] if pd.notnull(row['Question 17']) else None,
        row['Question 18'] if pd.notnull(row['Question 18']) else None,
        row['Question 19'] if pd.notnull(row['Question 19']) else None,
        row['Question 20'] if pd.notnull(row['Question 20']) else None,
        row['Question 21'] if pd.notnull(row['Question 21']) else None,
        row['Question 22'] if pd.notnull(row['Question 22']) else None,
        row['Question 23'] if pd.notnull(row['Question 23']) else None,
        row['Question 24'] if pd.notnull(row['Question 24']) else None,
        row['Question 25'] if pd.notnull(row['Question 25']) else None,
        row['Question 26'] if pd.notnull(row['Question 26']) else None,
        row['Question 27'] if pd.notnull(row['Question 27']) else None,
        row['Question 28'] if pd.notnull(row['Question 28']) else None,
        row['Question 29'] if pd.notnull(row['Question 29']) else None,
        row['Question 30'] if pd.notnull(row['Question 30']) else None,
        row['Question 31'] if pd.notnull(row['Question 31']) else None,
        row['Question 32'] if pd.notnull(row['Question 32']) else None,
        row['Question 33'] if pd.notnull(row['Question 33']) else None,
        row['Question 34'] if pd.notnull(row['Question 34']) else None,
        row['Question 35'] if pd.notnull(row['Question 35']) else None,
        row['Question 36'] if pd.notnull(row['Question 36']) else None,
        row['Question 37'] if pd.notnull(row['Question 37']) else None,
        row['Question 38'] if pd.notnull(row['Question 38']) else None,
        row['Question 39'] if pd.notnull(row['Question 39']) else None,
        row['Question 40'] if pd.notnull(row['Question 40']) else None,
        row['Question 41'] if pd.notnull(row['Question 41']) else None,
        row['Question 42'] if pd.notnull(row['Question 42']) else None,
        row['Question 43'] if pd.notnull(row['Question 43']) else None,
        row['Question 44'] if pd.notnull(row['Question 44']) else None,
        row['Question 45'] if pd.notnull(row['Question 45']) else None,
        row['Submission IP'] if pd.notnull(row['Submission IP']) else None,
        pd.to_datetime(row['Last Update Date'], errors='coerce').date() if pd.notnull(row['Last Update Date']) else None
    )
    conn = get_db_connection()
    if conn is None:
        return
    try:
        with conn:
            with conn.cursor() as cur:
                cur.execute(query, data)
    except Exception as e:
        print(f"Error inserting data: {e}")

# Iterate over the rows in the DataFrame and insert data into the tables
df = pd.read_csv('raw_data_edited.csv')
for index, row in df.iterrows():
    # insert_user_info(row)
    # print(f"Inserted user info for index: {index}")
    insert_assessment_entries(row)
    print(f"Inserted assessment entries for index: {index}")


Inserted assessment entries for index: 0
Inserted assessment entries for index: 1
Inserted assessment entries for index: 2
Inserted assessment entries for index: 3
Inserted assessment entries for index: 4
Inserted assessment entries for index: 5
Inserted assessment entries for index: 6
Inserted assessment entries for index: 7
Inserted assessment entries for index: 8
Inserted assessment entries for index: 9
Inserted assessment entries for index: 10
Inserted assessment entries for index: 11
Inserted assessment entries for index: 12
Inserted assessment entries for index: 13
Inserted assessment entries for index: 14
Inserted assessment entries for index: 15
Inserted assessment entries for index: 16
Inserted assessment entries for index: 17
Inserted assessment entries for index: 18
Inserted assessment entries for index: 19
Inserted assessment entries for index: 20
Inserted assessment entries for index: 21
Inserted assessment entries for index: 22
Inserted assessment entries for index: 23
In

In [5]:
from sshtunnel import SSHTunnelForwarder
import csv

def get_db_connection():
    server = SSHTunnelForwarder(
        ('letsrise.myonline.works', 22),  # SSH server and port
        ssh_username='ubuntu',  # SSH username
        ssh_pkey='~/.ssh/id_letsrise',  # Path to your private key
        remote_bind_address=('localhost', 5432)  # Database server and port
    )

    server.start()

    conn = psycopg2.connect(
        dbname='letsrise_v1',
        user='letsrise_intern',
        password='letsrise',
        host='localhost',
        port=server.local_bind_port  # Use the dynamically assigned local port
    )
    return conn, server

In [8]:
!pip3 install sshtunnel

## letsrise server connection

In [7]:
from sshtunnel import SSHTunnelForwarder

def get_db_connection():
    server = SSHTunnelForwarder(
        ('letsrise.myonline.works', 22),  # SSH server and port
        ssh_username='ubuntu',  # SSH username
        ssh_pkey='~/.ssh/id_letsrise',  # Path to your private key
        remote_bind_address=('localhost', 5432)  # Database server and port
    )

    server.start()

    conn = psycopg2.connect(
        dbname='letsrise_v1',
        user='letsrise_intern',
        password='letsrise',
        host='localhost',
        port=server.local_bind_port  # Use the dynamically assigned local port
    )
    return conn, server

In [14]:
def create_table(conn):
    with conn.cursor() as cur:
        cur.execute("""
            CREATE TABLE IF NOT EXISTS consequence_results (
                user_id UUID,
                name VARCHAR(255),
                delayed_product_market_fit FLOAT,
                lack_of_product_market_fit FLOAT,
                unable_to_complete_fundraise FLOAT,
                lack_of_growth FLOAT,
                lack_of_revenue FLOAT,
                high_turnover_of_talent FLOAT,
                inefficient_processes FLOAT,
                time_consuming_client_acquisition FLOAT,
                low_customer_conversion FLOAT,
                low_customer_satisfaction FLOAT,
                low_customer_retention FLOAT,
                high_cash_burnrate FLOAT,
                high_team_conflict FLOAT,
                high_key_man_risk FLOAT,
                lack_of_partnerships_collaborations FLOAT,
                lack_of_scalability FLOAT,
                lack_of_data_integrity FLOAT,
                lack_of_data_security FLOAT,
                lack_of_marketing FLOAT,
                lack_of_motivation FLOAT,
                lack_of_leadership FLOAT,
                lack_of_innovation FLOAT,
                lack_of_clarity FLOAT,
                too_much_dependency_on_external_factors FLOAT,
                lack_of_technological_advancements FLOAT,
                lack_of_unique_value_proposition FLOAT,
                lack_of_customer_variety FLOAT,
                lack_of_intellectual_property FLOAT,
                lacking_solution_quality FLOAT,
                lack_of_supporters FLOAT,
                missed_opportunities FLOAT,
                delayed_revenue FLOAT
            );
        """)
        conn.commit()

def insert_values_from_csv(conn, csv_file):
    with open(csv_file, 'r') as file:
        reader = csv.DictReader(file)
        with conn.cursor() as cur:
            for row in reader:
                cur.execute("""
                    INSERT INTO consequence_results (
                        user_id, name, delayed_product_market_fit, lack_of_product_market_fit, 
                        unable_to_complete_fundraise, lack_of_growth, lack_of_revenue, 
                        high_turnover_of_talent, inefficient_processes, 
                        time_consuming_client_acquisition, low_customer_conversion, 
                        low_customer_satisfaction, low_customer_retention, high_cash_burnrate, 
                        high_team_conflict, high_key_man_risk, lack_of_partnerships_collaborations, 
                        lack_of_scalability, lack_of_data_integrity, lack_of_data_security, 
                        lack_of_marketing, lack_of_motivation, lack_of_leadership, lack_of_innovation, 
                        lack_of_clarity, too_much_dependency_on_external_factors, 
                        lack_of_technological_advancements, lack_of_unique_value_proposition, 
                        lack_of_customer_variety, lack_of_intellectual_property, lacking_solution_quality, 
                        lack_of_supporters
                    ) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
                """, (
                    row['User_id'], row['Username'], 
                    float(row['Delayed Product Market Fit']), float(row['Lack of Product Market Fit']), 
                    float(row['Unable to Complete Fundraise']), float(row['Lack of Growth']), 
                    float(row['Lack of Revenue']), float(row['High Turnover of Talent']), 
                    float(row['Inefficient Processes']), float(row['Time Consuming Client Acquisition']), 
                    float(row['Low Customer Conversion']), float(row['Low Customer Satisfaction']), 
                    float(row['Low Customer Retention']), float(row['High Cash Burnrate']), 
                    float(row['High Team Conflict']), float(row['High Key-Man Risk']), 
                    float(row['Lack of Parternships/Collaborations']), float(row['Lack of Scalability']), 
                    float(row['Lack of Data Integrity']), float(row['Lack of Data Security']), 
                    float(row['Lack of Marketing']), float(row['Lack of Motivation']), 
                    float(row['Lack of Leadership']), float(row['Lack of Innovation']), 
                    float(row['Lack of Clarity']), float(row['Too much Dependancy on External Factors']), 
                    float(row['Lack of Technological Advancements']), float(row['Lack of Unique Value Proposition']), 
                    float(row['Lack of Customer Variety']), float(row['Lack of Intellectual Property']), 
                    float(row['Lacking Solution Quality']), float(row['Lack of Supporters'])
                ))
            conn.commit()

conn, server = get_db_connection()
try:
    create_table(conn)
    insert_values_from_csv(conn, 'scores.csv')
finally:
    conn.close()
    server.stop()